In [1]:
# 데이터 로드
import pandas as pd

train = pd.read_csv('C:/Users/sport/OneDrive/바탕 화면/VSC/train.csv')
valid = pd.read_csv('C:/Users/sport/OneDrive/바탕 화면/VSC/valid.csv')

In [2]:
import pandas as pd

train_clickbait_0 = train[train['clickbaitClass'] == 0].sample(n=40000, random_state=42)
train_clickbait_1 = train[train['clickbaitClass'] == 1].sample(n=40000, random_state=42)
train_sampled = pd.concat([train_clickbait_0, train_clickbait_1], ignore_index=True)

valid_clickbait_0 = valid[valid['clickbaitClass'] == 0].sample(n=10000, random_state=42)
valid_clickbait_1 = valid[valid['clickbaitClass'] == 1].sample(n=10000, random_state=42)
valid_sampled = pd.concat([valid_clickbait_0, valid_clickbait_1], ignore_index=True)

# 데이터 확인
print(train_sampled['clickbaitClass'].value_counts())
print(valid_sampled['clickbaitClass'].value_counts())

clickbaitClass
0    40000
1    40000
Name: count, dtype: int64
clickbaitClass
0    10000
1    10000
Name: count, dtype: int64


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 추출
X_train = train_sampled['newsContent'].tolist()
y_train = train_sampled['clickbaitClass'].tolist()
X_val = valid_sampled['newsContent'].tolist()
y_val = valid_sampled['clickbaitClass'].tolist()

In [9]:
# TF-IDF 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

# 텍스트 데이터를 TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)


In [10]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 학습
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)

# 검증 데이터에 대한 예측
y_val_pred = lr_model.predict(X_val_tfidf)

In [11]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# 혼동 행렬 계산
cm = confusion_matrix(y_val, y_val_pred)
cm

# 혼동 행렬을 비율로 변환
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_normalized

# 정확도, F1 스코어, 재현율 등의 지표 출력
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=['Non-Clickbait', 'Clickbait']))


Classification Report:
               precision    recall  f1-score   support

Non-Clickbait       0.64      0.62      0.63     10000
    Clickbait       0.63      0.66      0.65     10000

     accuracy                           0.64     20000
    macro avg       0.64      0.64      0.64     20000
 weighted avg       0.64      0.64      0.64     20000



In [12]:
# 새로운 기사 예측 함수
def predict_article_lr(article):
    article_tfidf = vectorizer.transform([article])
    prediction = lr_model.predict(article_tfidf)[0]
    return "bait" if prediction == 0 else "Nonbait"

# 새로운 기사 예측
new_article = "경찰은 불법촬영 신고 건수가 늘어나고 있다면서도 정확한 기준선이 없어 난감하다는 입장이다. 관련 업무를 담당하는 한 경찰 관계자는 의사에 반해 특정 부위 등을 반복해 촬영한 것이면 당연히 불법행위가 되지만 단순히 길거리를 촬영한 영상이면 현장에서 판단이 어려운 것은 사실이라며 사안에 따라 성범죄 전과 등 종합적으로 판단해 수사하고 있다고 설명했다. 법조계는 이같은 영상 촬영이 사전 동의 없이 이뤄지거나 특정 인물을 부각해 반복적으로 찍었다면 불법 소지가 있다고 경고한다. 지난 1월 대법원은 버스에서 레깅스 차림의 여성을 8초가량 촬영한 남성을 일상복을 입었다고 해 성적 욕망의 대상이 아니라고 단정할 수는 없다며 유죄로 판단한 바 있다. 의사에 반해 성적 대상화 우려가 있는 촬영은 불법이라는 의미다. 김기윤 변호사는 특정인물의 얼굴 역시 개인정보에 해당하기 때문에 이를 찍어 공개한 것은 개인정보보호법 위반에 해당할 소지가 있다며 특정 여성의 신체부위가 노출된 영상이라면 얼굴 노출 여부와 촬영 구도 등을 종합적으로 따져 성폭력범죄의 처벌 등에 관한 특례법 위반(카메라 등을 이용한 촬영)이 적용된다고 판단할 수 있다고 지적했다."
prediction = predict_article_lr(new_article)
print(f'Prediction for the new article: {prediction}')

Prediction for the new article: Nonbait
